# Homework 3

### The Cabbage Model

In Homework 3, we will explore a simple cabbage model originally created in Python, now adapted to the Cropbox framework. The phenology system is shown below. Although the code might seem complex, don't worry; we will only use the concepts and code we've learned in the course.

<div class="alert alert-warning">
If figures/tables are not rendered correctly, make sure your notebook is `trusted` by choosing `Trust Notebook` command found in `View` - `Activate Command Palette` menu. Typing in the first letters (i.e. "tr") should give you the command.
</div>

Load the necessary packages.

In [ ]:
using Cropbox

In [ ]:
using DataFrames
using CSV

In [ ]:
Cropbox.Interact.WebIO.setup(:ijulia)

We will use the corn weather file from **L04-Phenology** for the daily temperature in the model's input variables.

In [ ]:
corn_weather = CSV.read("corn_weather.csv", DataFrame) |> unitfy

### Homework questions: 
1. Let's create a mix-in system for loading up temperature variable from an external data source. Please fill in the missing codes in the code cell below.

In [ ]:
@system Temperature begin
    calendar(context)             ~      ::Calendar
    t(calendar.date): date        ~ track::date

    data                          ~ #= .. =#
    Ta:                temperature ~ #= .. =#
end

This is the model that depicts the phenological changes in the growth of cabbage.

In [ ]:
@system Phenology(Temperature, Controller) begin

    "plant density per m2"
    plantDensity ~ preserve(parameter, u"m^-2")

    "initial leaf number, default"
    initialLeafNumber ~ preserve(parameter)

    "days for root adaptation after transplanting"
    daysRoot ~ preserve(parameter, u"d")

    dap(context.clock.time) ~ track(u"d")
    
    # parameters for leaf number increasing
    Rxleaf => 1.838 ~ preserve(parameter, u"d^-1")
    Tnleaf => 0     ~ preserve(parameter, u"°C")
    Txleaf => 34.7  ~ preserve(parameter, u"°C")
    Toleaf => 20.57 ~ preserve(parameter, u"°C")

    "requirement sum of temp_rate for flowering"
    satTemp => 54 ~ preserve(parameter)

    "requirement sum of temp_rate for harvesting"
    satRep => 150 ~ preserve(parameter)

    "max. temp for veg. and rep, growth"
    optTemp => 10 ~ preserve(parameter, u"°C")

    "sum of ver_rate for flowering"
    satVer => 55  ~ preserve(parameter)

    "opt. temp. for vernalization"
    optVer => 7.0 ~ preserve(parameter, u"°C")

    midRateLN(Ta, Tn = Tnleaf, Tx = Txleaf, To = Toleaf, Rx = Rxleaf) => begin
        T = clamp(Ta, 0u"°C", Tx)
        Rx * ((Tx - T) / (Tx - To)) * ((T - Tn) / (To - Tn))^((To - Tn) / (Tx - To))
    end ~ track(u"d^-1")

    earlyRateLN(midRateLN) => begin
        0.5midRateLN
    end ~ track(u"d^-1")

    rateVer(nounit(Ta, u"°C"), nounit(optVer, u"°C")) => begin
        T = max(Ta, 0.01)
        exp(-log(T / optVer)^4)
    end ~ track(u"d^-1")

    sumVer(rateVer) ~ accumulate

    verdvs(sumVer, satVer) => begin
        sumVer / satVer
    end ~ track(max = 1)

    rateTemp(nounit(Ta, u"°C"), nounit(optTemp, u"°C")) => begin
        T = max(Ta, 0.01)
        exp(-log(T / optTemp)^2)
    end ~ track(u"d^-1")

    sumTemp(rateTemp) ~ accumulate

    tempdvs(sumTemp, satTemp, satRep) => begin
        if sumTemp >= satRep
            2
        elseif sumTemp > satTemp
            1 + (sumTemp - satTemp) / (satRep - satTemp)
        else
            sumTemp / satTemp
        end
    end ~ track

    leafRate(dap, daysRoot, earlyRateLN, midRateLN, pLeafForm) => begin
        if dap < daysRoot
            earlyRateLN
        else
            midRateLN * pLeafForm
        end
    end ~ track(u"d^-1")

    leafAcc(leafRate) ~ accumulate(init = initialLeafNumber)

    LN(leafAcc): leafNumber ~ track::int(round = :floor)

    dvs(tempdvs, verdvs) => begin
        tempdvs * verdvs
    end ~ track

    # from facade.py
    bolting(dvs) => begin
        !(dvs < 1)
    end ~ flag

    initial_pLeafForm ~ preserve(parameter)

    "suppression parameter for leaf formation rate"
    pLeafForm(initial_pLeafForm) ~ track
end

2. Please create a configuration that uses the weather data from `July 1st 2002` to `July 30th 2002`.

In [ ]:
phenology_config_summer = @config(
    Phenology => (;
        plantDensity      = 3.8,
        initialLeafNumber = 6,
        daysRoot          = 10,
        initial_pLeafForm = 0.69,
        
        optTemp = 10,
        optVer = 7,
        Rxleaf = 1.838,
        Tnleaf = 0,
        Txleaf = 34,
        Toleaf = 20,
    ),
    #= .. =#
)

3. Please create a configuration that uses the weather data from `Aug. 31st 2002` to `Sep 29th 2002`.

In [ ]:
phenology_config_autumn = @config(
    Phenology => (;
        plantDensity      = 3.8,
        initialLeafNumber = 6,
        daysRoot          = 10,
        initial_pLeafForm = 0.69,
        
        optTemp = 10,
        optVer = 7,
        Rxleaf = 1.838,
        Tnleaf = 0,
        Txleaf = 34,
        Toleaf = 20,
    ),
    #= .. =#
)

4. Please compare the simulation results of leaf number for the summer and autumn configurations.

5. Do the results match your expectations? Why are there differences in leaf number between the two seasons? You can discuss this in terms of plant physiology, parameter settings, crop characteristics, etc.

**Please upload your answer to NTU Cool system. Accepted format: .docx .pdf .ipynb .jl .zip .tar.gz**